In [4]:
import pandas as pd
import numpy as np
from ast import literal_eval
from tqdm.notebook import tqdm

In [2]:
tqdm.pandas()
def clean_data(data):
    # filter out the noise in the data
    data['domains'] = data['domains'].progress_apply(literal_eval)
    data = data[data['domains'].progress_apply(lambda x: 'gamereactor' not in x)]
    data = data[data['domains'].progress_apply(lambda x: 'duke' not in x)]
    return data

In [ ]:
path = "E:\\PycharmProject\\ada_project\\data\\"

years = [2015, 2016, 2017, 2018, 2019, 2020]  # 
all_occur = []
all_speaker = []
all_quote = []
for year in years:
    print(year)
    df = pd.read_csv(path + 'processed-quotes-{}.csv'.format(year))
    df = clean_data(df)
    gender_data = df.groupby('gender')
    quote_num = gender_data.apply(len)
    gender_freq = gender_data.agg({'numOccurrences': 'sum'}).sort_values(by='numOccurrences',
                                                                         ascending=False)
    
    speaker_df = df.drop_duplicates(subset='speaker')
    speaker_data = speaker_df.groupby('gender')
    
    speaker_num = speaker_data.apply(len)
    
    all_quote.append(quote_num)
    all_occur.append(gender_freq)
    all_speaker.append(speaker_num)
    
all_quote = pd.concat(all_quote, axis=1)
all_occur = pd.concat(all_occur, axis=1)
all_speaker = pd.concat(all_speaker, axis=1)
all_speaker

In [6]:
all_speaker = all_speaker.fillna(0)
sum_all_speaker = all_speaker.apply(sum, axis=1).sort_values(ascending=False)
sum_all_speaker['other'] = sum(sum_all_speaker.iloc[2:])
sum_all_speaker = sum_all_speaker[sum_all_speaker >= sum_all_speaker['other']]
sum_all_speaker

gender
male      1771658.0
female     526565.0
other        3277.0
dtype: float64

In [7]:
all_quote = all_quote.fillna(0)
sum_all_quote = all_quote.apply(sum, axis=1).sort_values(ascending=False)
sum_all_quote['other'] = sum(sum_all_quote.iloc[2:])
sum_all_quote = sum_all_quote[sum_all_quote >= sum_all_quote['other']]
sum_all_quote

gender
male      61509042.0
female    12704175.0
other       144659.0
dtype: float64

In [9]:
all_occur = all_occur.fillna(0)
sum_occur = all_occur.apply(sum, axis=1)
sum_occur['other'] = sum(sum_occur.iloc[2:])
sum_all_occur = sum_occur[sum_occur >= sum_occur['other']]
sum_all_occur

gender
male      228674655.0
female     48346221.0
other        577029.0
dtype: float64

In [11]:
all_freq = pd.concat([sum_all_speaker, sum_all_quote, sum_all_occur], axis=1)
all_freq.columns = ['speaker','quote', 'count']
all_freq

,speaker,quote,count
gender,,,
male,1771658.0,61509042.0,228674655.0
female,526565.0,12704175.0,48346221.0
other,3277.0,144659.0,577029.0


In [12]:
all_freq.to_csv('all_gender_freq.csv')